In [15]:
from openai import OpenAI

In [16]:
ollama_url = "http://localhost:11434/v1"
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [17]:
# Geopolitics Game
import json

class GeopoliticsGame:
    """
    A game where 5 LLM players (representing countries) compete for territories.
    Each player starts with 10 territories and has a strategic attitude (e.g., "Aggressive", 
    "Defensive", "Neutral", "Weak", "Strong", "Opportunistic") that influences their behavior.
    On their turn, a player proposes to take 1 territory from another player. All players 
    vote Yes or No. If more Yes votes, the territory transfer succeeds. 
    Goal: have the most territories after X turns.
    """
    
    def __init__(self, player_models, country_names, attitudes, ollama_client, starting_territories=10):
        """
        Initialize the game with 5 players.
        
        Args:
            player_models: List of 5 model names (e.g., ["phi3:medium", "llama2", ...])
            country_names: List of 5 country names (e.g., ["Avalon", "Borealis", ...])
            attitudes: List of 5 attitudes (e.g., ["Aggressive", "Defensive", "Neutral", ...])
            ollama_client: The Ollama OpenAI client instance
            starting_territories: Starting territories per player (default: 10)
        """
        if len(player_models) != 5:
            raise ValueError("Must have exactly 5 players")
        if len(country_names) != 5:
            raise ValueError("Must have exactly 5 country names")
        if len(attitudes) != 5:
            raise ValueError("Must have exactly 5 attitudes")
        
        self.player_models = player_models
        self.country_names = country_names
        self.attitudes = attitudes
        self.ollama = ollama_client
        self.starting_territories = starting_territories
        
        # Game state
        self.territories = [starting_territories] * 5  # Each player starts with 10
        self.history = []  # Full history of all turns
        self.total_tokens = 0
        
        # Create player info dictionary
        self.players = [
            {"id": i, "model": model, "country": country, "attitude": attitude}
            for i, (model, country, attitude) in enumerate(zip(player_models, country_names, attitudes))
        ]
    
    def build_game_state_context(self, player_id):
        """
        Build a context string showing current game state from a player's perspective.
        
        Args:
            player_id: The ID of the player (0-4)
        """
        player = self.players[player_id]
        context = f"You are {self.country_names[player_id]}, representing Player {player_id + 1}.\n"
        context += f"Your attitude and strategic approach: {player['attitude']}\n\n"
        context += "Current territory counts:\n"
        for i, (country, territories) in enumerate(zip(self.country_names, self.territories)):
            if i == player_id:
                context += f"  {country} (YOU): {territories} territories\n"
            else:
                context += f"  {country}: {territories} territories\n"
        
        return context
    
    def build_history_context(self, include_speeches=False, max_turns=None):
        """
        Build a context string from all previous turns.
        Excludes speeches to save context window - only includes basic turn outcomes.
        
        Args:
            include_speeches: If True, include speeches (default: False to save context)
            max_turns: Maximum number of recent turns to include (None = all turns)
        """
        if not self.history:
            return ""
        
        # Only show recent turns if max_turns is specified
        history_to_show = self.history
        if max_turns and len(self.history) > max_turns:
            history_to_show = self.history[-max_turns:]
            context = f"\n\nPrevious turns summary (showing last {max_turns} of {len(self.history)} turns, speeches excluded):\n"
        else:
            context = "\n\nPrevious turns summary (speeches excluded to save space):\n"
        
        for turn_data in history_to_show:
            turn_num = turn_data['turn']
            proposer = self.country_names[turn_data['proposer_id']]
            target = self.country_names[turn_data['target_id']]
            votes = turn_data['votes']
            passed = turn_data['passed']
            
            # Count yes/no votes
            yes_count = turn_data.get('yes_count', 0)
            no_count = turn_data.get('no_count', 0)
            
            context += f"Turn {turn_num}: {proposer} → {target}. "
            context += f"Vote: {yes_count}-{no_count}. "
            
            if passed:
                context += f"PASSED.\n"
            else:
                context += f"FAILED.\n"
            
            # Only include speeches if explicitly requested (for current turn only)
            if include_speeches:
                if 'proposal_speech' in turn_data and turn_data['proposal_speech']:
                    context += f"  {proposer}'s speech: \"{turn_data['proposal_speech']}\"\n"
                for i, (country, vote_data) in enumerate(zip(self.country_names, votes)):
                    if isinstance(vote_data, dict) and vote_data.get('speech'):
                        context += f"  {country}'s speech: \"{vote_data['speech']}\"\n"
        
        return context
    
    def _get_attitude_instruction(self, attitude, context_type, proposer=None, target=None):
        """
        Generate instruction text based on player attitude.
        
        Args:
            attitude: The player's attitude (e.g., "Aggressive", "Defensive", etc.)
            context_type: Either "proposal" or "vote"
            proposer: Name of proposer (for vote context)
            target: Name of target (for vote context)
        """
        attitude_lower = attitude.lower()
        
        if context_type == 'proposal':
            if 'aggressive' in attitude_lower or 'aggressive' == attitude_lower:
                return "Given your Aggressive attitude, you should be bold and target the strongest or most threatening opponent. Take initiative and don't hesitate to make ambitious proposals."
            elif 'defensive' in attitude_lower or 'defensive' == attitude_lower:
                return "Given your Defensive attitude, you should be cautious. Consider targeting players who have been aggressive towards you or who pose the greatest threat to your security."
            elif 'weak' in attitude_lower or 'weak' == attitude_lower:
                return "Given your Weak attitude, you should be careful and strategic. Target the weakest players to minimize risk, or try to avoid making enemies by targeting those who are already being targeted."
            elif 'strong' in attitude_lower or 'strong' == attitude_lower:
                return "Given your Strong attitude, you have confidence and power. You can afford to be strategic - target players who threaten your dominance or who are gaining too much power."
            elif 'neutral' in attitude_lower or 'neutral' == attitude_lower:
                return "Given your Neutral attitude, you should be balanced and pragmatic. Make decisions based on the current game state and what makes strategic sense."
            elif 'opportunistic' in attitude_lower or 'opportunistic' == attitude_lower:
                return "Given your Opportunistic attitude, you should look for the best strategic opportunities. Target players who are vulnerable or when the timing is advantageous."
            else:
                return f"Given your {attitude} attitude, make your proposal based on your strategic approach and the current game situation."
        
        elif context_type == 'vote':
            if 'aggressive' in attitude_lower or 'aggressive' == attitude_lower:
                return f"Given your Aggressive attitude, you generally support proposals that change the status quo. Consider voting Yes if it weakens strong opponents or creates opportunities for you."
            elif 'defensive' in attitude_lower or 'defensive' == attitude_lower:
                return f"Given your Defensive attitude, you should be cautious about voting. Vote Yes if it protects your interests or weakens threats to you, but vote No if it makes you more vulnerable."
            elif 'weak' in attitude_lower or 'weak' == attitude_lower:
                return f"Given your Weak attitude, you need to be very careful. Vote Yes if it helps you or weakens stronger players, but vote No if it makes you a target or if you're uncertain."
            elif 'strong' in attitude_lower or 'strong' == attitude_lower:
                return f"Given your Strong attitude, you can afford to be strategic. Vote Yes if it maintains or increases your advantage, vote No if it threatens your position."
            elif 'neutral' in attitude_lower or 'neutral' == attitude_lower:
                return f"Given your Neutral attitude, evaluate this proposal objectively. Vote based on what makes strategic sense given the current game state."
            elif 'opportunistic' in attitude_lower or 'opportunistic' == attitude_lower:
                return f"Given your Opportunistic attitude, vote Yes if this proposal creates opportunities for you or weakens competitors, vote No if it doesn't benefit you."
            else:
                return f"Given your {attitude} attitude, vote based on your strategic approach. Consider whether this proposal helps or harms your position."
        
        return ""
    
    def get_proposal(self, player_id):
        """
        Query a player to propose which country to take a territory from.
        
        Args:
            player_id: The ID of the proposing player (0-4)
            
        Returns:
            target_id: The ID of the target country (0-4, not equal to player_id)
        """
        player = self.players[player_id]
        game_state = self.build_game_state_context(player_id)
        # Limit history to last 10 turns to save context window
        history = self.build_history_context(include_speeches=False, max_turns=10)
        
        attitude_instruction = self._get_attitude_instruction(player['attitude'], 'proposal')
        
        prompt = f"""{game_state}{history}

{attitude_instruction}

It is your turn. You must propose to take 1 territory from one of the other countries.
You can also make a speech to try to influence other players' votes.

Respond with a JSON object containing:
- "Proposal": the country name you want to target (e.g., "Avalon" or "Borealis")
- "Speech": a persuasive speech to influence other players' votes (can be empty string if you don't want to speak)

Example format:
{{"Proposal": "Avalon", "Speech": "Avalon has been growing too powerful. We must act now to maintain balance."}}
"""
        
        messages = [{"role": "user", "content": prompt}]
        
        try:
            response = self.ollama.chat.completions.create(
                model=player["model"],
                messages=messages
            )
            response_text = response.choices[0].message.content.strip()
            
            # Track token usage
            if hasattr(response, 'usage') and response.usage:
                tokens_used = getattr(response.usage, 'total_tokens', 0)
                self.total_tokens += tokens_used
            
            # Try to parse JSON
            try:
                # Try to extract JSON from the response (in case it's wrapped in markdown or text)
                if "```json" in response_text:
                    response_text = response_text.split("```json")[1].split("```")[0].strip()
                elif "```" in response_text:
                    response_text = response_text.split("```")[1].split("```")[0].strip()
                
                proposal_data = json.loads(response_text)
                proposal_country = proposal_data.get("Proposal", "").strip()
                speech = proposal_data.get("Speech", "").strip()
            except (json.JSONDecodeError, KeyError) as e:
                # Fallback: try to extract country name from text
                print(f"Warning: Could not parse JSON from {player['country']}: {e}")
                print(f"  Response was: {response_text[:100]}...")
                proposal_country = response_text
                speech = ""
            
            # Find matching country name
            proposal_lower = proposal_country.lower()
            target_id = None
            for i, country in enumerate(self.country_names):
                if i != player_id and (country.lower() in proposal_lower or proposal_lower in country.lower()):
                    target_id = i
                    break
            
            # If no match found, default to first available country
            if target_id is None:
                print(f"Warning: Could not parse proposal '{proposal_country}' from {player['country']}, defaulting to first available")
                for i in range(5):
                    if i != player_id:
                        target_id = i
                        break
            
            if target_id is None:
                target_id = (player_id + 1) % 5  # Fallback
            
            return target_id, speech
            
        except Exception as e:
            print(f"Error querying {player['country']}: {e}")
            # Default to first available country
            for i in range(5):
                if i != player_id:
                    return i, ""
            return (player_id + 1) % 5, ""
    
    def get_vote(self, player_id, proposer_id, target_id, proposal_speech=""):
        """
        Query a player to vote Yes or No on a proposal.
        
        Args:
            player_id: The ID of the voting player (0-4)
            proposer_id: The ID of the player making the proposal (0-4)
            target_id: The ID of the target country (0-4)
            proposal_speech: The speech made by the proposer (optional)
            
        Returns:
            Dictionary with "vote" and "speech" keys
        """
        player = self.players[player_id]
        proposer = self.country_names[proposer_id]
        target = self.country_names[target_id]
        game_state = self.build_game_state_context(player_id)
        # Limit history to last 10 turns to save context window
        history = self.build_history_context(include_speeches=False, max_turns=10)
        
        attitude_instruction = self._get_attitude_instruction(player['attitude'], 'vote', proposer, target)
        
        proposal_text = f"{proposer} has proposed to take 1 territory from {target}."
        if proposal_speech:
            proposal_text += f"\n{proposer}'s speech: \"{proposal_speech}\""
        
        prompt = f"""{game_state}{history}

{attitude_instruction}

{proposal_text}
Do you vote Yes or No on this proposal?
You can also make a speech to try to influence other players' votes.

Respond with a JSON object containing:
- "Vote": either "Yes" or "No"
- "Speech": a persuasive speech to influence other players (can be empty string if you don't want to speak)

Example format:
{{"Vote": "Yes", "Speech": "I agree with this proposal. {target} has been too aggressive."}}
"""
        
        messages = [{"role": "user", "content": prompt}]
        
        try:
            response = self.ollama.chat.completions.create(
                model=player["model"],
                messages=messages
            )
            response_text = response.choices[0].message.content.strip()
            
            # Track token usage
            if hasattr(response, 'usage') and response.usage:
                tokens_used = getattr(response.usage, 'total_tokens', 0)
                self.total_tokens += tokens_used
            
            # Try to parse JSON
            try:
                # Try to extract JSON from the response (in case it's wrapped in markdown or text)
                if "```json" in response_text:
                    response_text = response_text.split("```json")[1].split("```")[0].strip()
                elif "```" in response_text:
                    response_text = response_text.split("```")[1].split("```")[0].strip()
                
                vote_data = json.loads(response_text)
                vote = vote_data.get("Vote", "No").strip()
                speech = vote_data.get("Speech", "").strip()
            except (json.JSONDecodeError, KeyError) as e:
                # Fallback: try to extract vote from text
                print(f"Warning: Could not parse JSON from {player['country']}: {e}")
                print(f"  Response was: {response_text[:100]}...")
                response_lower = response_text.lower()
                if "yes" in response_lower or "vote yes" in response_lower:
                    vote = "Yes"
                else:
                    vote = "No"
                speech = ""
            
            # Normalize the vote
            vote_lower = vote.lower()
            if "yes" in vote_lower:
                vote = "Yes"
            else:
                vote = "No"
            
            return {"vote": vote, "speech": speech}
            
        except Exception as e:
            print(f"Error querying {player['country']}: {e}")
            return {"vote": "No", "speech": ""}  # Default to No on error
    
    def play_turn(self, turn_num):
        """
        Play a single turn of the game.
        
        Args:
            turn_num: The turn number
        """
        # Determine whose turn it is (round-robin: turn 1 = player 0, turn 2 = player 1, etc.)
        proposer_id = (turn_num - 1) % 5
        proposer = self.players[proposer_id]
        
        print(f"\n{'='*60}")
        print(f"Turn {turn_num}: {proposer['country']}'s turn")
        print(f"{'='*60}")
        
        # Get proposal
        print(f"Querying {proposer['country']} ({proposer['model']}) for proposal...")
        target_id, proposal_speech = self.get_proposal(proposer_id)
        target = self.country_names[target_id]
        print(f"{proposer['country']} proposes to take 1 territory from {target}")
        if proposal_speech:
            print(f"  {proposer['country']}'s speech: \"{proposal_speech}\"")
        
        # Get votes from all players
        print(f"\nCollecting votes...")
        votes = []
        for i in range(5):
            vote_data = self.get_vote(i, proposer_id, target_id, proposal_speech)
            votes.append(vote_data)
            vote_str = vote_data.get('vote', 'No') if isinstance(vote_data, dict) else vote_data
            print(f"  {self.country_names[i]}: {vote_str}")
            if isinstance(vote_data, dict) and vote_data.get('speech'):
                print(f"    {self.country_names[i]}'s speech: \"{vote_data['speech']}\"")
        
        # Count votes
        yes_count = sum(1 for v in votes if (v.get('vote', 'No') if isinstance(v, dict) else v) == "Yes")
        no_count = sum(1 for v in votes if (v.get('vote', 'No') if isinstance(v, dict) else v) == "No")
        passed = yes_count > no_count
        
        # Execute territory transfer if passed
        if passed:
            if self.territories[target_id] > 0:
                self.territories[proposer_id] += 1
                self.territories[target_id] -= 1
                print(f"\n✓ Proposal PASSED ({yes_count} Yes, {no_count} No)")
                print(f"  {proposer['country']} now has {self.territories[proposer_id]} territories")
                print(f"  {target} now has {self.territories[target_id]} territories")
            else:
                print(f"\n✗ Proposal PASSED but {target} has no territories to lose!")
        else:
            print(f"\n✗ Proposal FAILED ({yes_count} Yes, {no_count} No)")
            print(f"  No territory transfer")
        
        # Store turn data
        turn_data = {
            'turn': turn_num,
            'proposer_id': proposer_id,
            'target_id': target_id,
            'proposal_speech': proposal_speech,
            'votes': votes,
            'yes_count': yes_count,
            'no_count': no_count,
            'passed': passed,
            'territories_after': self.territories.copy()
        }
        self.history.append(turn_data)
        
        # Display current standings
        print(f"\nCurrent standings:")
        for i, (country, territories) in enumerate(zip(self.country_names, self.territories)):
            print(f"  {country}: {territories} territories")
    
    def play_game(self, num_turns):
        """
        Play a full game with the specified number of turns.
        
        Args:
            num_turns: Number of turns to play
        """
        print(f"\n{'='*60}")
        print(f"Starting Geopolitics Game")
        print(f"{'='*60}")
        print(f"\nPlayers:")
        for i, player in enumerate(self.players):
            print(f"  Player {i+1}: {player['country']} ({player['model']}) - {player['attitude']}")
        print(f"\nStarting territories per player: {self.starting_territories}")
        print(f"Total turns: {num_turns}")
        print(f"{'='*60}")
        
        # Reset game state
        self.territories = [self.starting_territories] * 5
        self.history = []
        self.total_tokens = 0
        
        # Play turns
        for turn_num in range(1, num_turns + 1):
            self.play_turn(turn_num)
        
        # Final summary
        self.print_summary()
    
    def print_summary(self):
        """Print a summary of the game."""
        print(f"\n{'='*60}")
        print("FINAL RESULTS")
        print(f"{'='*60}")
        
        # Sort players by territories
        standings = sorted(
            [(self.country_names[i], self.territories[i], i) for i in range(5)],
            key=lambda x: x[1],
            reverse=True
        )
        
        print(f"\nFinal standings:")
        for rank, (country, territories, player_id) in enumerate(standings, 1):
            print(f"  {rank}. {country}: {territories} territories")
        
        winner = standings[0]
        print(f"\n🏆 Winner: {winner[0]} with {winner[1]} territories!")
        
        print(f"\nTurn-by-turn summary:")
        for turn_data in self.history:
            proposer = self.country_names[turn_data['proposer_id']]
            target = self.country_names[turn_data['target_id']]
            result = "PASSED" if turn_data['passed'] else "FAILED"
            print(f"  Turn {turn_data['turn']}: {proposer} → {target} ({result}, {turn_data['yes_count']}-{turn_data['no_count']})")
        
        print(f"\nTotal tokens used: {self.total_tokens}")

In [18]:
# Create and play the Geopolitics game
# You need 5 models, 5 country names, and 5 attitudes
# Each player starts with 10 territories (default)
# Players take turns proposing to take territories from others
# All players vote Yes/No, and if more Yes votes, the transfer succeeds
# Attitudes influence how players make proposals and vote (e.g., "Aggressive", "Defensive", "Neutral", "Weak", "Strong", "Opportunistic")

game = GeopoliticsGame(
    player_models=[
        "phi3:medium",    # Change to your preferred models
        "phi3:medium",
        "phi3:medium",
        "phi3:medium",
        "phi3:medium"
    ],
    country_names=["Avalon", "Borealis", "Crestfall", "Dawnhold", "Elysium"],  # Change country names if you want
    attitudes=["Aggressive", "Defensive", "Neutral", "Opportunistic", "Weak"],  # Each player's strategic attitude
    ollama_client=ollama,
    starting_territories=10  # Each player starts with 10 territories
)

# Play 5 rounds (you can change this number)
game.play_game(num_turns=10)


Starting Geopolitics Game

Players:
  Player 1: Avalon (phi3:medium) - Aggressive
  Player 2: Borealis (phi3:medium) - Defensive
  Player 3: Crestfall (phi3:medium) - Neutral
  Player 4: Dawnhold (phi3:medium) - Opportunistic
  Player 5: Elysium (phi3:medium) - Weak

Starting territories per player: 10
Total turns: 10

Turn 1: Avalon's turn
Querying Avalon (phi3:medium) for proposal...
Avalon proposes to take 1 territory from Crestfall
  Avalon's speech: "As a leader, I have always stressed on the importance of balance and equity. Crestfall's recent expansion is tipping that balance in their favor. By reducing their territories now, we can ensure fair play for all."

  Avalon: Yes
    Avalon's speech: "As Avalon, I have always prided myself on maintaining equity in our game of territories. Currently, Crestfall's growth is unchecked and gives them an unfair advantage. By proposing to shift one territory from them, we level the playing field and ensure a more competitive environment for

Interesting - if the players can talk, no territory is taken.